<a href="https://colab.research.google.com/github/alemskdlt/dl02/blob/main/project/cifar10/cifar10_DL3B_TL_tf_datasets_EfficientNetB1_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning
- cifar10 dataset from tensorflow_datasets
- ConvNet: EfficientNetB1

> https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB1

In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np

tf.__version__

## Load cifar10 using tensorflow_datasets

In [ ]:
# CFAR-10 데이터 세트를 적재한다. (tensorflow_datasets)
import tensorflow_datasets as tfds

Batch_size = 64
# 
dataset_name = "cifar10"  # change the name of the dataset 
# PrefetchDataset : BatchDataSet => (None, 32, 32, 3)
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, 
    split=["train", "test"], 
    with_info=True, 
    batch_size=Batch_size,  # preset mini-batch
    as_supervised=True
)

NUM_CLASSES = ds_info.features["label"].num_classes
print(NUM_CLASSES)
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print(ds_train)  # PrefetchDataset
# PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3)
# Batch preset?

for i, (image, label) in enumerate(ds_train.take(1)):
    print(i, image.shape, label)


In [ ]:
print(ds_train)
print(ds_info)
ds_info.features

In [ ]:
# Show samples
# _ = tfds.show_examples(ds_train, ds_info)
IMG_SIZE = 128 #120 #240  # for Transfer Learning using EfficientNetB1
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

print(len(ds_train),len(ds_test))

str(ds_train)  # MapDataset
for i, (image, label) in enumerate(ds_train.take(1)):
    print(i, image.shape, label)


In [ ]:
50000/64,10000/64

In [ ]:
str(ds_train)

In [ ]:
#
# Visualizing the dataset
#
# The following code shows the first 9 images with their labels.

print("="*25, 'Train dataset', "="*25)
# figure 크기를 조절합니다.
plt.figure(figsize=(6, 6))
# 배치 하나를 가져옵니다.
for images, labels in ds_train.take(1):    # Make a batch of images & labels
    print(images.shape)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # tensor2numpy array: tensor.numpy()
        # plt.title(label_names[int(labels[i])])
        plt.title(str(labels[i].numpy()) + ", " + label_names[int(labels[i])])
        plt.axis("off")
plt.show()

print("="*25, 'Test dataset', "="*25)

plt.figure(figsize=(6, 6))
for images, labels in ds_test.take(1):    # Make a batch of images & labels
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        # plt.title(label_names[int(labels[i])])
        plt.title(str(labels[i].numpy()) + ", " + label_names[int(labels[i])])
        plt.axis("off")
plt.show()


## Data augmentation

In [ ]:
#
# Data augmentation
#
from tensorflow.keras import layers
tf.get_logger().setLevel('ERROR')  # Clear warnings in data augmentation
# Create a data augmentation with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(factor=0.1),
  layers.RandomZoom(0.1),
  # layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
  layers.RandomHeight(0.1),
  layers.RandomWidth(0.1),
  # layers.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

# Plot the augmented images
plt.figure(figsize=(6,6))
image_idx = np.random.randint(10)   # 0~ 9
for images, labels in ds_train.take(1):    # Make a batch of images & labels
    print(labels,images.shape)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        aug_img = data_augmentation(tf.expand_dims(images[image_idx], axis=0))
        print(aug_img.shape)
        plt.imshow(aug_img[0].numpy().astype("uint8"))
        plt.title("{}".format(label_names[labels[image_idx]]))
        plt.axis("off")
    break
plt.show()

plt.figure(figsize=(6,6))
image_idx = np.random.randint(10)   # 0~ 9
for images, labels in ds_test.take(1):    # Make a batch of images & labels
    print(labels,images.shape)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        aug_img = data_augmentation(tf.expand_dims(images[image_idx], axis=0))
        print(aug_img.shape)
        plt.imshow(aug_img[0].numpy().astype("uint8"))
        plt.title("{}".format(label_names[labels[image_idx]]))
        plt.axis("off")
    break
plt.show()


# Model: Transfer learning
- ## EfficientNet V1 : EfficientNetB1
---
> ### EfficientNet-B1은 EfficientNetV1 계열의 모델 중 하나로, Compound Scaling 방법을 사용해 기초 모델인 EfficientNet-B0에 비해 성능과 효율성이 향상된 모델입니다.  ( from wrtn.ai )

### EfficientNet-B1의 주요 특징은 다음과 같습니다:

1. **가변성**: EfficientNet-B1은 기본 이미지 해상도를 240x240으로 지원하며, 따라서 이보다 크거나 작은 이미지를 처리하기에도 적합합니다.
2. **상대적으로 작은 모델**: 이 모델은 실전 상황에서 쉽게 배포할 수 있는 모델을 제공하며, 컴퓨터 자원에 큰 부담을 주지 않습니다. 그럼에도 불구하고, 비슷한 크기의 다른 모델에 비해 더 높은 성능을 보입니다.
3. **복합 스케일링**: EfficientNet-B1은 모델의 깊이, 너비, 그리고 해상도를 각각 조절하는 구조로, 이미지 입력 크기와 파라미터 수를 유연하게 조정할 수 있도록 설계되어 있습니다.
4. **사전 훈련 가중치**: EfficientNet-B1은 ImageNet 데이터셋에서 이미 학습된 가중치를 제공합니다. 이렇게 전이학습에 사용할 수 있는 사전 훈련된 가중치가 있기 때문에 비교적 적은 양의 데이터셋으로부터도 높은 성능의 모델을 학습할 수 있습니다.

- **EfficientNet-B1은 다양한 이미지 분류 작업에 효과적으로 사용될 수 있으며, 소량의 컴퓨팅 파워와 메모리를 사용하여도 높은 성능을 낼 수 있습니다.**
- **전이학습에 유용한 이 모델은 다양한 사례에서 활용될 수 있어, 인기 있는 모델 중 하나** 입니다.
---
> EfficientNetV2는 EfficientNet에서 개선된 모델 계열로서, 합성곱신경망의 성능과 크기의 균형을 더욱 잘 맞추도록 설계되었습니다. EfficientNetV2 모델 계열에는 학습 및 구현을 위한 여러 가지 사이즈와 구성의 모델이 포함되어 있습니다.

## EfficientNetV2 모델 계열에는 다음과 같은 모델들이 있습니다:

- EfficientNetV2-B0: 가장 기본 모델로, 파라미터 수가 약 55M 미만입니다. 이 모델은 기본 디자인을 시작점으로 삼아 이후에 나오는 모델들의 성능과 크기를 조절합니다.
- EfficientNetV2-B1: 약 78M 개의 파라미터를 가진 중간 크기의 모델로, 입력 이미지의 크기는 240×240입니다.
- EfficientNetV2-B2: 약 90M 개의 파라미터를 가진 조금 더 큰 모델로, 입력 이미지의 크기는 260×260입니다.
- EfficientNetV2-B3: 약 122M 개의 파라미터를 가진 크고 강력한 모델로, 입력 이미지의 크기는 300×300입니다.
- EfficientNetV2S: 여러 모델 중 가장 작은 모델로, 입력 이미지의 크기가 224×224이고, 모델 파라미터의 수는 약 22M입니다.
- EfficientNetV2M: 중간 크기의 모델로, 약 305M 개의 파라미터를 가지고 있으며 입력 이미지의 크기는 112×112입니다.
- EfficientNetV2L: 중간 크기의 모델보다 큰 모델로, 약 400M 개의 파라미터를 가지고 있으며 입력 이미지의 크기는 128×128입니다.
> 각 EfficientNetV2 모델은 다양한 요구사항과 자원에 맞게 적합한 크기와 성능을 가지고 있습니다. 이 모델들은 기존의 EfficientNet 계열보다 높은 성능과 효율성을 달성하며, 다양한 작업에서 좋은 결과를 낼 수 있습니다. 그리고, EfficientNetV2 모델들은 이미지넷에서 미리 훈련된 가중치를 가지고 있으므로, 전이 학습에서도 높은 성능을 보입니다.

In [ ]:
#
# Transfer learning => Fine Tuning
#
## Using the model EfficientNetB1 for the first experiment with all the layers trainable 
## Creating the model 

from tensorflow.keras.applications.efficientnet import EfficientNetB1
# base_model = EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='max')
base_model = tf.keras.applications.EfficientNetB1(include_top = False, weights='imagenet')
base_model.trainable = True # Full Training

# Fine-tune from this layer onwards
fine_tune_at = 235  # half of the whole layers

#  Fine-tuning after layer_number larger than 235
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name, layer.trainable, end=", ")
  

model = tf.keras.Sequential([
  layers.Input(shape=(IMG_SIZE,IMG_SIZE,3),name='input_layer'),
  # layers.Rescaling(1./255),
  data_augmentation,
  # Fine Tuning
  base_model,
  layers.GlobalMaxPooling2D(name = "global_max"),
  # FCN
  layers.Dense(128,activation='relu'),
  layers.Dense(10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])


model.summary()


In [ ]:
# Check the input and output of base_model
ix=layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))
print(base_model(ix))

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

## Building the Model

In [ ]:
# Building the Model
# Inspecting the train_data
ds_train
# Setting up the callbacks
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 5 epochs in a row, stop training
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.25, # multiply the learning rate by 0.2 (reduce by 4x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

## Check the summary
for no, layer in enumerate(model.layers):
  print(no, layer.trainable)



##  Training model using augmentated dataset

In [ ]:
#
######################################################
# Training model using augmentated data
######################################################
#
%%time
history = model.fit(ds_train, 
                    epochs=100, 
                    steps_per_epoch = len(ds_train), 
                    validation_data = ds_test,
                    validation_steps = len(ds_test), # batchSize,
                    callbacks = [early_stopping, reduce_lr])

#
model.evaluate(ds_test)
# loss: 0.1658 - accuracy: 0.9595



## Plot of learning curves
- loss, val_loss
- accuracy, val_accuracy

In [ ]:
# 손실값을 그래프로 그린다. 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc = 'lower right')
plt.show()

# 정확도를 그래프로 그린다. 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'], loc = 'lower right')
plt.show()

#############################################
# More training graphs
# More graphs of loss and accuracy
# import matplotlib.pyplot as plt
# import numpy as np

history_dict = history.history 
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.figure(figsize=(14, 4))

plt.subplot(1,2,1)
plt.plot(epochs, loss, 'go-', label='Training Loss')
plt.plot(epochs, val_loss, 'bd', label='Validation Loss')
plt.plot(np.argmin(np.array(val_loss))+1,val_loss[np.argmin(np.array(val_loss))], 'r*', ms=12)
plt.title('Training and Validation Loss, min: ' + str(np.round(val_loss[np.argmin(np.array(val_loss))],4)))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

epochs = range(1, len(loss) + 1)

plt.subplot(1,2,2)
plt.plot(epochs, acc, 'go-', label='Training Accuracy') #, c='blue')
plt.plot(epochs, val_acc, 'bd', label='Validation Accuracy') #, c='red')
plt.plot(np.argmax(np.array(val_acc))+1,val_acc[np.argmax(np.array(val_acc))], 'r*', ms=12)
plt.title('Training and Validation Accuracy, max: ' + str(np.round(val_acc[np.argmax(np.array(val_acc))],4)))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(6, 6))
for images, labels in ds_test.take(1):  # Make a batch of images & labels
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(str(labels[i].numpy()) + ", " + label_names[int(labels[i])])
        plt.axis("off")
plt.show()



## Evaluation using test dataset

In [ ]:
y_pred0 = model.predict(ds_test)
y_pred = np.argmax(y_pred0, axis=1)
y_test = [labels.numpy() for _, labels in ds_test.unbatch()]
print("정답=", y_test[0])
print("예측값=", y_pred[0], np.argmax(y_pred0[0]))

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
# 0.9595

#################################
# Evaluate the model
#################################
model.evaluate(ds_test)  #,y_test)
# loss: 1.1689 - accuracy: 0.6468
# loss: 0.9207 - accuracy: 0.6799    # with dropout: 0.5
# loss: 0.1658 - accuracy: 0.9595    # Transfer learning using EfficientNetB1
# [0.20311422646045685, 0.9466000199317932]


In [ ]:
# Save the trained model
model.save('cifar10-TL-EfficientNetB1-Fine-Tuning.hdf5')